In [1]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn
import copy

c:\Users\ADMIN\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Uniform Sample**


In [9]:
class UniformSampleFrames:
    """Uniformly sample frames from the video.
    To sample an n-frame clip from the video. UniformSampleFrames basically
    divide the video into n segments of equal length and randomly sample one
    frame from each segment. To make the testing results reproducible, a
    random seed is set during testing, to make the sampling results
    deterministic.
    Required keys are "total_frames", "start_index" , added or modified keys
    are "frame_inds", "clip_len", "frame_interval" and "num_clips".
    Args:
        clip_len (int): Frames of each sampled output clip.
        num_clips (int): Number of clips to be sampled. Default: 1.
        test_mode (bool): Store True when building test or validation dataset.
            Default: False.
        seed (int): The random seed used during test time. Default: 255.
    """

    def __init__(self, clip_len, num_clips=1, test_mode=False, seed=255):

        self.clip_len = clip_len
        self.num_clips = num_clips
        self.test_mode = test_mode
        self.seed = seed

    def _get_train_clips(self, num_frames, clip_len):
        """Uniformly sample indices for training clips.
        Args:
            num_frames (int): The number of frames.
            clip_len (int): The length of the clip.
        """

        assert self.num_clips == 1
        if num_frames < clip_len:
            start = np.random.randint(0, num_frames)
            inds = np.arange(start, start + clip_len)
        elif clip_len <= num_frames < 2 * clip_len:
            basic = np.arange(clip_len)
            inds = np.random.choice(
                clip_len + 1, num_frames - clip_len, replace=False)
            offset = np.zeros(clip_len + 1, dtype=np.int64)
            offset[inds] = 1
            offset = np.cumsum(offset)
            inds = basic + offset[:-1]
        else:
            bids = np.array(
                [i * num_frames // clip_len for i in range(clip_len + 1)])
            bsize = np.diff(bids)
            bst = bids[:clip_len]
            offset = np.random.randint(bsize)
            inds = bst + offset
        return inds

    def _get_test_clips(self, num_frames, clip_len):
        """Uniformly sample indices for testing clips.
        Args:
            num_frames (int): The number of frames.
            clip_len (int): The length of the clip.
        """

        np.random.seed(self.seed)
        if num_frames < clip_len:
            # Then we use a simple strategy
            if num_frames < self.num_clips:
                start_inds = list(range(self.num_clips))
            else:
                start_inds = [
                    i * num_frames // self.num_clips
                    for i in range(self.num_clips)
                ]
            inds = np.concatenate(
                [np.arange(i, i + clip_len) for i in start_inds])
        elif clip_len <= num_frames < clip_len * 2:
            all_inds = []
            for i in range(self.num_clips):
                basic = np.arange(clip_len)
                inds = np.random.choice(
                    clip_len + 1, num_frames - clip_len, replace=False)
                offset = np.zeros(clip_len + 1, dtype=np.int64)
                offset[inds] = 1
                offset = np.cumsum(offset)
                inds = basic + offset[:-1]
                all_inds.append(inds)
            inds = np.concatenate(all_inds)
        else:
            bids = np.array(
                [i * num_frames // clip_len for i in range(clip_len + 1)])
            bsize = np.diff(bids)
            bst = bids[:clip_len]
            all_inds = []
            for i in range(self.num_clips):
                offset = np.random.randint(bsize)
                all_inds.append(bst + offset)
            inds = np.concatenate(all_inds)
        return inds

    def __call__(self, results):
        num_frames = results['total_frames']

        if self.test_mode:
            inds = self._get_test_clips(num_frames, self.clip_len)
        else:
            inds = self._get_train_clips(num_frames, self.clip_len)

        inds = np.mod(inds, num_frames)
        # start_index = results['start_index']
        start_index = 0
        inds = inds + start_index

        results['frame_inds'] = inds.astype(np.int64)
        results['clip_len'] = self.clip_len
        results['frame_interval'] = None
        results['num_clips'] = self.num_clips
        return results

    def __repr__(self):
        repr_str = (f'{self.__class__.__name__}('
                    f'clip_len={self.clip_len}, '
                    f'num_clips={self.num_clips}, '
                    f'test_mode={self.test_mode}, '
                    f'seed={self.seed})')
        return repr_str


In [6]:
datalst = pds.read_pickle('Data/Test1.pkl')


In [7]:
a = datalst[0]

In [10]:
UniSample = UniformSampleFrames(100)
result = UniSample(a)


In [12]:
result['frame_inds']

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
      dtype=int64)

In [3]:
# class UniformSampleFrames:
#     """Uniformly sample frames from the video.

#     To sample an n-frame clip from the video. UniformSampleFrames basically
#     divide the video into n segments of equal length and randomly sample one
#     frame from each segment. To make the testing results reproducible, a
#     random seed is set during testing, to make the sampling results
#     deterministic.

#     Required keys are "total_frames", "start_index" , added or modified keys
#     are "frame_inds", "clip_len", "frame_interval" and "num_clips".

#     Args:
#         clip_len (int): Frames of each sampled output clip.
#         num_clips (int): Number of clips to be sampled. Default: 1.
#         test_mode (bool): Store True when building test or validation dataset.
#             Default: False.
#         seed (int): The random seed used during test time. Default: 255.
#     """

#     def __init__(self, clip_len, num_clips=1, test_mode=False, seed=255):

#         self.clip_len = clip_len
#         self.num_clips = num_clips
#         self.test_mode = test_mode
#         self.seed = seed

#     def _get_train_clips(self, num_frames, clip_len):
#         """Uniformly sample indices for training clips.

#         Args:
#             num_frames (int): The number of frames.
#             clip_len (int): The length of the clip.
#         """

#         assert self.num_clips == 1
#         if num_frames < clip_len:
#             start = np.random.randint(0, num_frames)
#             inds = np.arange(start, start + clip_len)
#         elif clip_len <= num_frames < 2 * clip_len:
#             basic = np.arange(clip_len)
#             inds = np.random.choice(
#                 clip_len + 1, num_frames - clip_len, replace=False)
#             offset = np.zeros(clip_len + 1, dtype=np.int64)
#             offset[inds] = 1
#             offset = np.cumsum(offset)
#             inds = basic + offset[:-1]
#         else:
#             bids = np.array(
#                 [i * num_frames // clip_len for i in range(clip_len + 1)])
#             bsize = np.diff(bids)
#             bst = bids[:clip_len]
#             offset = np.random.randint(bsize)
#             inds = bst + offset
#         return inds

#     def _get_test_clips(self, num_frames, clip_len):
#         """Uniformly sample indices for testing clips.

#         Args:
#             num_frames (int): The number of frames.
#             clip_len (int): The length of the clip.
#         """

#         np.random.seed(self.seed)
#         if num_frames < clip_len:
#             # Then we use a simple strategy
#             if num_frames < self.num_clips:
#                 start_inds = list(range(self.num_clips))
#             else:
#                 start_inds = [
#                     i * num_frames // self.num_clips
#                     for i in range(self.num_clips)
#                 ]
#             inds = np.concatenate(
#                 [np.arange(i, i + clip_len) for i in start_inds])
#         elif clip_len <= num_frames < clip_len * 2:
#             all_inds = []
#             for i in range(self.num_clips):
#                 basic = np.arange(clip_len)
#                 inds = np.random.choice(
#                     clip_len + 1, num_frames - clip_len, replace=False)
#                 offset = np.zeros(clip_len + 1, dtype=np.int64)
#                 offset[inds] = 1
#                 offset = np.cumsum(offset)
#                 inds = basic + offset[:-1]
#                 all_inds.append(inds)
#             inds = np.concatenate(all_inds)
#         else:
#             bids = np.array(
#                 [i * num_frames // clip_len for i in range(clip_len + 1)])
#             bsize = np.diff(bids)
#             bst = bids[:clip_len]
#             all_inds = []
#             for i in range(self.num_clips):
#                 offset = np.random.randint(bsize)
#                 all_inds.append(bst + offset)
#             inds = np.concatenate(all_inds)
#         return inds

 

In [13]:
sample  = UniformSampleFrames(clip_len=100)
inds = sample._get_train_clips(54, 100)
print(inds)

[ 36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135]


In [14]:
inds = np.mod(inds, 54)
print(inds)

[36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53  0  1  2  3  4  5
  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29
 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]


## **Some Function**

In [ ]:
def Interpolation(kp, num_frames):
    """
    Repeat frames in a keypoints numpy array using linear interpolation.
    
    Args:
        kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing keypoints data.
        num_frames: integer representing the number of frames to interpolate to.
        
    Returns:
        sampled_kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing the interpolated keypoints.
    """
    # get the number of frames in the original keypoints array
    num_original_frames = kp.shape[0]
    
    # create a new array of shape (num_person, num_frames, num_keypoints, 3) to store the interpolated keypoints
    sampled_kp = np.zeros((num_frames, kp.shape[1], kp.shape[2]))
    
    # loop through each person in the keypoints array
    # loop through each keypoint in the keypoints array
    for kpt in range(kp.shape[1]):
        # loop through each coordinate in the keypoint (x, y, z)
        for coord in range(kp.shape[3]):
            # create an array of x values representing the original frames
            x = np.arange(num_original_frames)
            # create an array of x values representing the new frames
            new_x = np.linspace(0, num_original_frames-1, num_frames)
            # use linear interpolation to calculate the y values (keypoint coordinates) at the new frames
            new_y = np.interp(new_x, x, kp[:, kpt, coord])
            # store the interpolated keypoint coordinates in the new array
            sampled_kp[ :, kpt, coord] = new_y
                
    return sampled_kp

def repeat_frames(kp, num_frames):
  """
  Repeat frames in a keypoints numpy array to achieve uniform sampling.

  Args:
      kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing keypoints data.
      num_frames: integer representing the number of frames to sample to.
      
  Returns:
      sampled_kp: numpy array of shape (num_person, num_frames, num_keypoints, 3) containing the keypoints after repeating frames.
  Chua Fix kp
  """
  # check if the number of frames in the keypoints array is less than the desired number of frames
  if kp.shape[1] < num_frames:
      # calculate the number of frames to repeat
      num_frames_to_repeat = num_frames - kp.shape[1]
      # determine the indices of the frames to repeat
      repeat_indices = np.random.choice(kp.shape[1], num_frames_to_repeat)
      # sort the indices in ascending order
      repeat_indices = np.sort(repeat_indices)
      # create an array of zeros to store the sampled keypoints
      sampled_kp = np.zeros((kp.shape[0], num_frames, kp.shape[2], kp.shape[3]))
      # loop through each person in the keypoints array
      for person in range(kp.shape[0]):
          # loop through each keypoint in the keypoints array
          for kpt in range(kp.shape[2]):
              # loop through each coordinate in the keypoint (x, y, z)
              for coord in range(kp.shape[3]):
                  # create a new array of keypoint coordinates with repeated frames
                  new_kp = np.repeat(kp[person, :, kpt, coord][np.newaxis, :], num_frames, axis=0)
                  # repeat the frames at the specified indices using linear interpolation
                  new_kp[repeat_indices] = Interpolation(kp[person, :, kpt, coord][np.newaxis, :], num_frames_to_repeat)
                  # store the new keypoint coordinates in the sampled keypoints array
                  sampled_kp[person, :, kpt, coord] = new_kp
  else:
      # if the number of frames in the keypoints array is greater than or equal to the desired number of frames, return the keypoints array as is
      sampled_kp = kp

  return sampled_kp



# **Action Dataset**

In [ ]:
from copy import deepcopy
class ActionDataset(Dataset):
    def __init__(self,path, train_mode, clip_len=100):
        # super().__init__()
        # # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
        # # self.transform = Transform
        # self.feature=[]
        # self.label=[]
        # # self.append(Data,label)
        self.train_mode = train_mode
        self.file = pds.read_pickle(path)
        self.clip_len = clip_len
        self.sample = UniformSampleFrames(self.clip_len)
        

    def __getitem__(self, index):
        kp = deepcopy(self.file[index]['kp'][0])
        # score = deepcopy(self.file[index]['keypoint_score'][0])
        # score = np.expand_dims(score,axis = 2)
        # kp = np.concatenate((kp,score),axis=2)
        w, h = self.file[index]['img_shape']
        kp[:,:,0] = (kp[:,:,0]-w/2)/(w/2)
        kp[:,:,1] = (kp[:,:,1]-h/2)/(h/2)
        label = deepcopy(self.file[index]['label'])
        if self.train_mode:
            inds = self.sample._get_train_clips(len(kp), self.clip_len)
        else: 
            inds = self.sample._get_test_clips(len(kp), self.clip_len)
        start_index = 0
        inds = inds + start_index
        inds = np.mod(inds, len(kp))
        kp = kp[inds] 
            
        return torch.from_numpy(kp).float(), label
    
    def __len__(self):
        return len(self.file)
    
    # def SetTrans(self,Transform):
    #     self.transform=Transform
    
    # def append(self,Data):
    #     # super().__init__()
    #     # assert all(param is not None for param in [Data,label]),"Data and label must be give in"
    #     # assert label==None,"Label must be give"
    #     # kpscore = np.expand_dims(Data['keypoint_score'][0],axis=2)
    #     kp = Data['kp'][0]
    #     h, w = Data['img_shape']
    #     # shapeimg=Data['img_shape']
    #     ## normalize pic
    #     # kp[:,:,0] = kp[:,:,0]/w
    #     # kp[:,:,1] = kp[:,:,1]/h
    #     kp[:,:,0] = (kp[:,:,0]-w/2)/(w/2)
    #     kp[:,:,1] = (kp[:,:,1]-h/2)/(h/2)
    #     #############
    #     # data = np.concatenate((kp,kpscore),axis=2)
    #     # data = np.expand_dims(data,axis=0)
    #     label = Data['label']
    #     kp = torch.from_numpy(kp).float()
    #     label = torch.tensor(label).long()
    #     self.feature.append(kp)
    #     self.label.append(label)
    #     self.leng=len(self.feature)  

class ToTensor():
    def __call__(self,sample):
        data,label=sample
        return torch.from_numpy(data.astype(np.float32)),torch.tensor(label)

# **Train Model**

In [ ]:
def Train_epoch(model=None,loss_fn=None,train_loader=None,optimizer=None,device='cuda'):
    model.train()
    # model.training = True
    total_loss=0
    for index,(data,label) in enumerate(train_loader):
        data = data.to(device)
        outputs = model(data)
        label=label.to(device).long()
        loss = loss_fn(outputs,label)
        total_loss+=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    total_loss/=len(train_loader)
    return total_loss    

def Val_epoch(model=None,loss_fn=None,val_loader=None,device='cuda'):
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.eval()
    # model.to(device)
    total_loss = 0
    preds = []
    labels = []
    model.training = True
    with torch.no_grad():
        for index,(data,label) in enumerate(val_loader):
            data = data.to(device)
            outputs = model(data)
            label = label.to(device).long()
            loss = loss_fn(outputs,label)
            # pred = torch.argmax(outputs,dim=1)
            # labels.append(label.item())
            # preds.append(pred.item())
            total_loss+=loss
    total_loss/=len(val_loader)
    #acc multi class CrossEntropy
    # acc = eval_acc(preds,labels)
    return total_loss

def eval_acc(preds,labels):
    n_total = len(preds)
    print(n_total)
    n_correct = 0
    for pred,label in zip(preds,labels):
        if pred == label: n_correct+=1
        else: continue
    acc=n_correct/n_total
    return acc
        

## **TRAIN N EVAL**

In [ ]:
def Train_n_Eval(model:nn.Module=None,epochs=None,loss_fn=None,lr=1e-4,optim:torch.optim.Adam=None,
                 train_dataloader=None,eval_dataloader=None,lr_shedule=False,
                 Step=10,miles=2,Gamma=0.1,device='cuda'):
    assert all(param is not None for param in [model,epochs,loss_fn,optim,
                                               train_dataloader,eval_dataloader]),"All Param must be give in"
    device=torch.device(device if torch.cuda.is_available() else 'cpu')
    model.to(device)
    # optim = optim(model.parameters(),lr,momentum=0.9,nesterov=True)
    optim = optim(model.parameters(),lr)
    loss_history = {
        'train': [],
        'val' : [],
    }
    best_score=0
    if lr_shedule:
        Multistep=[Step * i for i in range(1,miles+1)]
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optim,Multistep,Gamma)
    for epoch in range(1,epochs+1):
        lr=optim.param_groups[-1]['lr']
        train_loss = Train_epoch(model,loss_fn,train_dataloader,optim)
        val_loss = Val_epoch(model,loss_fn,eval_dataloader)
        loss_history['train'].append(train_loss)
        loss_history['val'].append(val_loss)
        # if acc > best_score:
        #     best_score = acc
        #     model_best_wts = copy.deepcopy(model.state_dict())
        #     torch.save(model.state_dict(),'Model_best_wts.pt')
            # print("Copied best model weights!")
        if lr_shedule:
            scheduler.step()
        print(f'Epoch: {epoch}: Learning rate: {lr}\n \tTrain Loss: {train_loss}\n\tVal Loss: {val_loss}')
    model_final = copy.deepcopy(model.state_dict())
    torch.save(model.state_dict(),'model_final.pth')
    return model_final
        

## **Read File and Train**

In [ ]:
train_dataset = ActionDataset('Data/train1.pkl', train_mode=True, clip_len=100)
val_dataset = ActionDataset('Data/Valid1.pkl',train_mode=False, clip_len=100)

In [ ]:
model=ModelSTGCN(3,2)
criterion=nn.CrossEntropyLoss()
optim=torch.optim.Adam
# datalst1 = pds.read_pickle('Data/train1.pkl')
# datalst2 = pds.read_pickle('Data/Valid1.pkl')
train_dataset = ActionDataset('Data/train1.pkl', train_mode=True, clip_len=100)
val_dataset = ActionDataset('Data/Valid1.pkl',train_mode=False, clip_len=100)
# for data in datalst1:
#     train_dataset.append(data)
# for data in datalst2:
#     val_dataset.append(data)
train_dataloader=DataLoader(dataset=train_dataset, batch_size=30)
val_dataloader=DataLoader(dataset=val_dataset, batch_size=10)
model_best = Train_n_Eval(model=model,epochs=200,loss_fn=criterion,optim=optim,
                                      train_dataloader=train_dataloader,eval_dataloader=val_dataloader,
                                      lr=1e-3,lr_shedule=True,Step=15)

## **Keep Training**

In [ ]:
# model_best,model_final = Train_n_Eval(model=model,epochs=5,loss_fn=criterion,optim=optim.SGD,
#                                       train_dataloader=train_dataloader,eval_dataloader=val_dataloader,
#                                       lr=1e-3,lr_shedule=True,Step=4)

## **Eval Model**

In [ ]:
# torch.save(model_best,'model_best.pth')

In [ ]:
# # model.load_state_dict(model_best)
# model.eval()
# # filetest=pds.read_pickle('/content/Data/test.pkl')
# testdata=ActionDataset('/content/Data/test.pkl',train_mode=False, clip_len=30)
# # for data in filetest:
# #     testdata.append(data)
# test_loader=DataLoader(testdata,shuffle=True,batch_size = 10)
# total_loss = 0
# preds = []
# labels = []
# with torch.no_grad():
#     for index,(data,label) in enumerate(test_loader):
#         outputs = model(data.to('cuda'))
#         label = label.to('cuda')
#         loss = criterion(outputs,label)
#         pred = torch.argmax(outputs,dim=1)
#         # labels.append(label.item())
#         # preds.append(pred.item())
#         total_loss+=loss
# total_loss/=(index + 1)
# #acc multi class CrossEntropy
# # acc = eval_acc(preds,labels)
# print(f'Total Loss: {total_loss} ')